<a href="https://colab.research.google.com/github/Joao-vitor1/fine-tuning-colonial-pt/blob/main/gerador_respostas_gemma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# codigo final

## baixando bibliotecas

In [ ]:
!pip install -U "transformers" "datasets" "peft" "accelerate" "bitsandbytes" "trl"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GemmaTokenizer
from transformers import BatchEncoding, AutoConfig
from trl import SFTTrainer, SFTConfig
from huggingface_hub import notebook_login
import torch
import os
from torch import autocast
import gc

os.environ['WANDB_DISABLED'] = "true"

In [ ]:
notebook_login()

## Baixando modelo e tokenizador

In [ ]:
#Fine tunning
model_id = "google/gemma-2-9b-it"
#https://huggingface.co/google/gemma-27b-it

# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

# Load the pretrained model.................................................................
model = AutoModelForCausalLM.from_pretrained(model_id,
                                              quantization_config=bnb_config,
                                             device_map="auto")

# Load the tokenizer..............................................................................
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#model's token limit.........................................................................
config = AutoConfig.from_pretrained(model_id)
token_limit =config.max_position_embeddings
token_limit

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model

## baixando dataset

In [ ]:
from google.colab import files

df = pd.read_csv('/content/dataset_input.csv')
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle()

In [ ]:
df[df['Documento'].isin(['PP_10','DN078','CAM220'])]

In [ ]:
dataset_test = Dataset.from_pandas(df[df['Documento'].isin(['PP_10'])])
dataset_test.num_rows

In [ ]:
def get_instruction(data_point):
  instruction = f"""traduza o texto abaixo escrito em português antigo para um português mais contemporâneo. Me dê apenas a tradução do texto, sem recomendações ou notas das mudanças feitas. Me retorne a resposta apenas em português.
"""
# Algumas informações do texto são:
  # add_instruction = False
  # if data_point['Data']:
  #   instruction += f"\nData em que o texo foi escrito: {data_point['Data']}."
  #   add_instruction = True
  # if data_point['Séc']:
  #   instruction += f"\nSéculo em que o texto foi escrito: Século {data_point['Séc']}."
  #   add_instruction = True
  # if data_point['Lugar']:
  #   instruction += f"\nLugar em que o texto foi escrito: {data_point['Lugar']}."
  #   add_instruction = True
  # if data_point['Região']:
  #   instruction += f"\nRegião que o texto foi escrito: {data_point['Região']}."
  #   add_instruction = True
  # if data_point['Notário'] is not None:
  #   instruction += f"\nNotário do texto é {data_point['Notário']}"
  #   add_instruction = True
  # if add_instruction == False:
  #   instruction += f"\nSem informações adicionais para esse texto"

  return instruction

In [ ]:
def get_input(begin_tokenizado, texto_tokenizado,end_tokenizado):
  input_ids = []
  attention_mask = []
  for batch in range(texto_tokenizado['input_ids'].shape[0]):

    concatenated_input = torch.cat([begin_tokenizado['input_ids'].squeeze(0),
                                    texto_tokenizado['input_ids'][batch, 1:],
                                    end_tokenizado['input_ids'].squeeze(0)[1:]], dim=0)

    concatenated_mask = torch.cat([begin_tokenizado['attention_mask'].squeeze(0),
                                  texto_tokenizado['attention_mask'][batch, 1:],
                                  end_tokenizado['attention_mask'].squeeze(0)[1:]], dim=0)

    input_ids.append(concatenated_input)
    attention_mask.append(concatenated_mask)

  input_ids = torch.stack(input_ids, dim=0)
  attention_mask = torch.stack(attention_mask, dim=0)

  overflow = texto_tokenizado['overflow_to_sample_mapping']

  new_text = BatchEncoding(
      {
          "input_ids": input_ids,
          "attention_mask": attention_mask,
          "overflow_to_sample_mapping": overflow,
      }
  )
  return new_text

## Gerando traduções

In [ ]:
# def get_output(data_point, token_limit, device):
#   torch.cuda.empty_cache()
#   torch.cuda.reset_max_memory_allocated()
#   gc.collect()
#   os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#   instruction = get_instruction(data_point)

#   begin = f"<start_of_turn>user {instruction}\n O texto para ser traduzido é este abaixo:\n"
#   end = "<end_of_turn>\n <start_of_turn>model"
#   print('documento:', data_point['Documento'])

#   begin_tokenizado = tokenizer(begin, return_tensors="pt", add_special_tokens=True).to(device)
#   end_tokenizado = tokenizer(end, return_tensors="pt", add_special_tokens=True).to(device)

#   max_length = token_limit - (begin_tokenizado['input_ids'].shape[1] + end_tokenizado['input_ids'].shape[1])

#   stride = token_limit // 4
#   texto_tokenizado = tokenizer(data_point['input'], padding=True, return_overflowing_tokens=True,
#                                truncation=True, stride=stride, max_length= max_length,
#                                return_tensors="pt", add_special_tokens=True).to(device)

#   tokenized_input = get_input(begin_tokenizado, texto_tokenizado, end_tokenizado) # função que concatena o texto com as instruções
#   tokenized_input = {k: v.to("cuda:0") for k, v in tokenized_input.items()}
#   print('input_shape:', tokenized_input['input_ids'].shape)


#   with autocast("cuda"):
#     generated_ids = model.generate(input_ids = tokenized_input['input_ids'],
#                                 attention_mask = tokenized_input['attention_mask'],
#                                 max_new_tokens=100, do_sample=False, pad_token_id=tokenizer.eos_token_id)

#   size_input = tokenized_input['input_ids'].shape[1]

#   decoded_inputs = []
#   decoded_outputs = []

#   for generated_id in generated_ids:
#     input = generated_id[:size_input]
#     output = generated_id[size_input:]

#     decoded_inputs.append(tokenizer.decode(input, skip_special_tokens=False))
#     decoded_outputs.append(tokenizer.decode(output, skip_special_tokens=False))

#   return decoded_inputs, decoded_outputs

In [ ]:
def get_output(data_point, token_limit, device):
  torch.cuda.empty_cache()
  torch.cuda.reset_max_memory_allocated()
  gc.collect()
  os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

  instruction = get_instruction(data_point)

  begin = f"<start_of_turn>user {instruction}\n O texto para ser traduzido é este abaixo:\n"
  end = "<end_of_turn>\n <start_of_turn>model"
  print('documento:', data_point['Documento'])

  begin_tokenizado = tokenizer(begin, return_tensors="pt", add_special_tokens=True).to(device)
  end_tokenizado = tokenizer(end, return_tensors="pt", add_special_tokens=True).to(device)

  # max_length = token_limit - (1000 + begin_tokenizado['input_ids'].shape[1] + end_tokenizado['input_ids'].shape[1])
  max_length = 1500
  stride = max_length // 4
  texto_tokenizado = tokenizer(data_point['input'], padding=True, return_overflowing_tokens=True,
                               truncation=True, stride=stride, max_length= max_length,
                               return_tensors="pt", add_special_tokens=True).to(device)

  decoded_inputs = []
  decoded_outputs = []
  for batch in range(texto_tokenizado['input_ids'].shape[0]):

    concatenated_input = torch.cat([begin_tokenizado['input_ids'].squeeze(0),
                                    texto_tokenizado['input_ids'][batch, 1:],
                                    end_tokenizado['input_ids'].squeeze(0)[1:]], dim=0)

    concatenated_mask = torch.cat([begin_tokenizado['attention_mask'].squeeze(0),
                                  texto_tokenizado['attention_mask'][batch, 1:],
                                  end_tokenizado['attention_mask'].squeeze(0)[1:]], dim=0)

    tokenized_input = BatchEncoding(
      {
          "input_ids": concatenated_input.unsqueeze(0),
          "attention_mask": concatenated_mask.unsqueeze(0),
      }
    )

    tokenized_input = {k: v.to("cuda:0") for k, v in tokenized_input.items()}
    print('input_shape:', tokenized_input['input_ids'].shape)


    with autocast("cuda"):
      generated_ids = model.generate(input_ids = tokenized_input['input_ids'],
                                  attention_mask = tokenized_input['attention_mask'],
                                  max_new_tokens= 3*max_length, do_sample=False, pad_token_id=tokenizer.eos_token_id)

    size_input = tokenized_input['input_ids'].shape[1]



    for generated_id in generated_ids:
      input = generated_id[:size_input]
      output = generated_id[size_input:]

      decoded_inputs.append(tokenizer.decode(input, skip_special_tokens=False))
      decoded_outputs.append(tokenizer.decode(output, skip_special_tokens=False))
    # break

  return decoded_inputs, decoded_outputs

In [ ]:
entradas = []
saidas = []
device = "cuda:0"

cont = 0
for i in range(40):
  inputs, outputs = get_output(dataset[i], token_limit, device)
  entradas.extend(inputs)
  saidas.extend(outputs)
  cont += 1
  if cont % 10 == 0:
    dataset_output = pd.DataFrame({'input': entradas, 'output': saidas})
    dataset_output.to_csv('dataset_output.csv')
    files.download('/content/dataset_output.csv')

In [ ]:
print(entradas[0])

In [ ]:
print(saidas[0])